In [1]:
import sys
import os

REPO_PATH = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if REPO_PATH not in sys.path:
    sys.path.insert(0, REPO_PATH)

    
from llama_cpp import Llama
from src.prompts.prompts import SYSTEM_PROMPT, SYSTEM_PROMPT_CLASSIFICATION
from src.app.llm import generate_response
from src.app.classif import generate_response_classification

In [2]:
MODELS_CONFIG = {
    "mistral-7b": { # 4.3Go
        "path": "../models/mistral-7b-instruct-v0.2.Q4_K_M.gguf", # C'est la version conçue pour le chat et les instructions
    },
    "qwen2.5-1.5b": { # 1.6Go
        "path": "../models/qwen2.5-1.5b-instruct-q8_0.gguf", # Excellente performance, en particulier sur les langues non-anglaises.
    },
    "qwen2.5-7b": { # 1.6Go
        "path": "../models/qwen2.5-7b-instruct-q2_k.gguf", # Excellente performance, en particulier sur les langues non-anglaises.
    },
    "qwen3-2b": { # 1.6Go
        "path": "../models/Qwen3-1.7B-Q8_0.gguf", # Excellente performance, en particulier sur les langues non-anglaises.
    }
}

MODEL_PATH = MODELS_CONFIG["mistral-7b"]["path"]
MODEL_PATH = MODELS_CONFIG["qwen2.5-1.5b"]["path"]
MODEL_PATH = MODELS_CONFIG["qwen2.5-7b"]["path"]

llm = Llama(
    model_path=MODEL_PATH,
    n_ctx=2048, 
    chat_format="chatml",
    verbose=False 
)

llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64

In [3]:
def generate_response(question):
    """
    Fonction qui envoie la phrase de l'utilisateur au LLM et traite le résultat.
    Utilise une approche "few-shot" avec 4 exemples diversifiés pour guider le modèle.
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": question}
    ]

    response = llm.create_chat_completion(
        messages=messages,
        temperature=0.3,
        max_tokens=512
    )

    return response['choices'][0]['message']['content'].strip()

def generate_response_classification(question):
    """
    Fonction qui envoie la phrase de l'utilisateur au LLM et traite le résultat.
    Utilise une approche "few-shot" avec 4 exemples diversifiés pour guider le modèle.
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT_CLASSIFICATION},
        {"role": "user", "content": question}
    ]

    response = llm.create_chat_completion(
        messages=messages,
        temperature=0,
        max_tokens=10
    )

    return response['choices'][0]['message']['content'].strip()

In [4]:
list_questions_grave = ["J’ai mal à la gorge depuis deux jours, est-ce grave ?"]
    
for question in list_questions_grave:
    reponse = generate_response(question)
    proba = generate_response_classification(question)
    print(f"Question : {question}\nRéponse : {reponse}\nProba : {proba}\n")


Question : J’ai mal à la gorge depuis deux jours, est-ce grave ?
Réponse : Il est normal de ressentir une douleur à la gorge de temps en temps. Cela peut être causé par de simples irritations de la muqueuse de la gorge, souvent dues à la déshydratation, la fume ou l'exposition à des irritants comme la fumée ou le pollen.

Cependant, si votre gorge est très douloureuse, si vous avez des difficultés à manger ou à boire, ou si vous avez des pharyngites associées tels que des glandes sous la langue ou dans le cou qui sont gonflées, il est important de consulter un professionnel de la santé.

En général, ces symptômes peuvent être dus à des infections virales ou bactériennes, mais il est important de noter que seuls un professionnel de la santé peut diagnostiquer et traiter les infections.

Rappelez-vous, il est toujours préférable de consulter un médecin si vous avez des inquiétudes ou si vos symptômes persistent ou s'aggravent.
Proba : 0.2

